In [7]:
import serial
import time
from datetime import datetime
from openpyxl import Workbook, load_workbook

# Configurações
PORT = 'COM11'  # Substitua pela porta correta do seu Arduino
BAUD_RATE = 9600

# Configurações do controle de temperatura
SETPOINT = 120.0  # Define o setpoint da temperatura
HISTERESE = 2.0  # Define a faixa de histerese

# Nome do arquivo
FILE_NAME = 'dados_arduino.xlsx'

def verifica_setpoint(temperatura_corrigida):
    """
    Função para verificar o status do setpoint:
    Retorna:
    - Status do Setpoint: 'Não chegou ainda' ou 'Já passou'
    - Acionamento da Resistência: 1 (ligada) ou 0 (desligada)
    """
    if temperatura_corrigida < SETPOINT - HISTERESE:
        return "Não chegou ainda", 1  # Resistência ligada (1) quando abaixo do setpoint
    elif temperatura_corrigida > SETPOINT + HISTERESE:
        return "Já passou", 0  # Resistência desligada (0) quando acima do setpoint
    else:
        return "No Setpoint", 0  # Resistência desligada (0) quando dentro do setpoint

def configurar_conexao_serial(port, baud_rate):
    try:
        return serial.Serial(port, baud_rate)
    except serial.SerialException as e:
        print(f"Erro ao abrir a porta serial: {e}")
        return None

def carregar_arquivo_excel(file_name):
    try:
        return load_workbook(file_name)
    except FileNotFoundError:
        return Workbook()

def criar_aba(wb, sheet_index):
    sheet_name = f'Teste_{sheet_index}'
    return wb.create_sheet(title=sheet_name)

def processar_dados(ser, ws, wb):
    while True:
        try:
            linha = ser.readline().decode('utf-8').strip()
            if linha and "Temp. Bruta" in linha:
                print(f"Recebido: {linha}")
                partes = linha.split(',')
                temp_bruta = partes[0].split(':')[1].strip().split()[0]  # Temperatura Bruta
                temp_corrigida = float(partes[1].split(':')[1].strip().split()[0])  # Temperatura Corrigida
                temp_ambiente = partes[2].split(':')[1].strip().split()[0]  # Temperatura Ambiente
                
                hora_atual = datetime.now().strftime('%H:%M')
                
                status_setpoint, resistencia_status = verifica_setpoint(temp_corrigida)
                
                ws.append([hora_atual, temp_bruta, temp_corrigida, temp_ambiente, status_setpoint, resistencia_status])
                
                print(f"Hora: {hora_atual}, Temp Bruta: {temp_bruta} C, Temp Corrigida: {temp_corrigida} C, Temp Ambiente: {temp_ambiente} C, Status: {status_setpoint}, Resistência: {resistencia_status}")
                
                # Salva o arquivo Excel a cada iteração para garantir que os dados não se percam
                wb.save(FILE_NAME)
        except Exception as e:
            print(f"Erro ao ler dados: {e}")

def main():
    ser = configurar_conexao_serial(PORT, BAUD_RATE)
    if ser:
        time.sleep(2)  # Espera o Arduino inicializar
        
        wb = carregar_arquivo_excel(FILE_NAME)
        sheet_index = len(wb.sheetnames) + 1 if wb else 1
        ws = criar_aba(wb, sheet_index)
        ws.append(['Hora', 'Temperatura Bruta (C)', 'Temperatura Corrigida (C)', 'Temperatura Ambiente (C)', 'Status do Setpoint', 'Acionamento da Resistência (1=Ligada, 0=Desligada)'])
        
        print(f"Conectado à porta {PORT}. Começando a coletar dados...")
        
        processar_dados(ser, ws, wb)
        
        ser.close()

if __name__ == "__main__":
    main()


Erro ao abrir a porta serial: could not open port 'COM11': PermissionError(13, 'Acesso negado.', None, 5)


In [2]:
import pandas as pd
from datetime import datetime, timedelta

# Carrega a tabela de valores de y a partir de um arquivo .xlsx
# Certifique-se de que o arquivo tenha uma coluna chamada "y"
df = pd.read_excel("dados2.xlsx")  # Substitua "dados.xlsx" pelo nome do seu arquivo

# Obtém o número de dados em y
y = df["y"].tolist()
n = len(y)

# Define o horário inicial e final
start_time = datetime.strptime("15:27", "%H:%M")
end_time = datetime.strptime("16:45", "%H:%M")

# Calcula o intervalo de tempo entre cada ponto
time_interval = (end_time - start_time) / (n - 1)

# Gera a lista de horários para o eixo x
x = [start_time + i * time_interval for i in range(n)]

# Adiciona a coluna x à tabela
df["x"] = [time.strftime("%H:%M") for time in x]

# Exibe a tabela resultante
print(df)

# Salva a nova tabela em um novo arquivo .xlsx, caso queira guardar o resultado
df.to_excel("dados_com_x.xlsx", index=False)


       Unnamed: 0      y      x
0             NaN  56.39  15:27
1             NaN  57.10  15:27
2             NaN  56.86  15:27
3             NaN  56.86  15:27
4             NaN  57.10  15:27
...           ...    ...    ...
23343         NaN  73.75  16:44
23344         NaN  74.21  16:44
23345         NaN  73.98  16:44
23346         NaN  73.51  16:44
23347         NaN  73.51  16:44

[23348 rows x 3 columns]


C:\Users\bruno\AppData\Local\Temp\ipykernel_16340\1550114014.py:29: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df.to_excel("dados_com_x.xlsx", index=False)


In [5]:
!pip install openpyxl pd

Defaulting to user installation because normal site-packages is not writeable


In [6]:
pip install --upgrade openpyxl


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
